# PCA for Biomarker Discovery in Drug Development

This notebook explains **how PCA works**, **why each step is required**, and **how parameter choices affect analysis and plots**.

## Code: PCA with Detailed Syntax and Parameter Explanations

In [ ]:
# Import NumPy for numerical computations
# Used here for random number generation and array operations
import numpy as np

# Import Pandas for tabular data handling
# DataFrames allow labeled rows and columns, which is critical for gene interpretation
import pandas as pd

# Import plotting libraries
# matplotlib controls figure-level plotting
# seaborn provides higher-level statistical visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Import StandardScaler for feature scaling
# PCA is variance-based and is sensitive to feature magnitude
from sklearn.preprocessing import StandardScaler

# Import PCA for dimensionality reduction
from sklearn.decomposition import PCA

# -------------------------------
# Reproducibility control
# -------------------------------
# Setting a random seed ensures that random numbers
# (and therefore synthetic data) are identical across runs
np.random.seed(42)

# -------------------------------
# Dataset dimensions
# -------------------------------
# num_samples controls how many biological samples are simulated
# num_features controls how many genes/proteins are measured per sample
# Increasing num_features increases dimensionality and correlation complexity
num_samples = 50
num_features = 100

# -------------------------------
# Generate synthetic biomarker data
# -------------------------------
# np.random.rand generates values in [0, 1)
# Multiplying by 10 increases variance scale
# Larger variance leads to stronger principal components
data = np.random.rand(num_samples, num_features) * 10

# Convert to DataFrame with gene labels
# f-strings dynamically generate column names
df = pd.DataFrame(data, columns=[f'Gene_{i+1}' for i in range(num_features)])

# -------------------------------
# Standardize the data
# -------------------------------
# StandardScaler subtracts the mean and divides by standard deviation
# This ensures all genes contribute equally to PCA
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)

# -------------------------------
# Perform PCA
# -------------------------------
# n_components controls how many principal components are retained
# Increasing n_components retains more variance but reduces dimensionality less
pca = PCA(n_components=10)

# fit_transform computes principal axes and projects the data
principal_components = pca.fit_transform(scaled_data)

# Store PCA results in a DataFrame
pc_df = pd.DataFrame(
    principal_components,
    columns=[f'PC_{i+1}' for i in range(10)]
)

# -------------------------------
# Explained variance analysis
# -------------------------------
# explained_variance_ratio_ shows fraction of variance captured by each PC
# cumsum computes cumulative explained variance
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

# -------------------------------
# Scree plot
# -------------------------------
# figure() controls plot size in inches
plt.figure(figsize=(10, 6))

# Plot individual explained variance
# 'o-' specifies circular markers connected by lines
plt.plot(
    range(1, len(explained_variance_ratio) + 1),
    explained_variance_ratio,
    'o-',
    linewidth=2,
    label='Individual Explained Variance'
)

# Plot cumulative explained variance
plt.plot(
    range(1, len(explained_variance_ratio) + 1),
    cumulative_explained_variance,
    'o-',
    linewidth=2,
    label='Cumulative Explained Variance'
)

# Axis labels and title guide interpretation
plt.title('Explained Variance by Principal Components')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')

# legend() maps curves to their meaning
plt.legend()

# grid(True) improves readability of variance trends
plt.grid(True)
plt.show()

# -------------------------------
# PCA loadings for biomarker discovery
# -------------------------------
# components_ contains eigenvectors (loadings)
# Rows correspond to PCs, columns to genes
loadings = pca.components_

loading_df = pd.DataFrame(
    loadings,
    columns=df.columns,
    index=[f'PC_{i+1}' for i in range(pca.n_components)]
)

# Identify genes with strongest influence on PC1
# abs() ignores sign and focuses on magnitude of contribution
top_5_genes_pc1 = loading_df.loc['PC_1'].abs().sort_values(ascending=False).head(5)

print("\nTop 5 potential biomarkers based on PC1 loadings:")
print(top_5_genes_pc1)

# -------------------------------
# Heatmap of PCA loadings
# -------------------------------
# Heatmaps reveal global contribution patterns across genes and PCs
plt.figure(figsize=(12, 8))
sns.heatmap(loading_df.T, cmap='viridis', annot=False)
plt.title('PCA Loadings Heatmap')
plt.xlabel('Principal Components')
plt.ylabel('Genes')
plt.show()